In [44]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row

In [45]:
# Importing the required notebooks
%run DifferentialEvolution.ipynb
%run IslandMethod.ipynb

In [46]:
# Start spark
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('IPDE') \
    .getOrCreate()
sc = spark.sparkContext
        
# Setting the test
diffEvo = DifferentialEvolution()
island = IslandMethod(diffEvo = diffEvo)

# Executing the island
island.execute(sc)


Current iteration: 0
Indexes: [3, 2, 0, 3, 2, 2, 2, 0, 0, 3, 1, 2, 1, 1, 3, 1, 0, 0, 1, 3]
Number of partitions: 4

Partitions structure: [[(0, array([-2.11758786,  0.62821643,  3.31601353])), (0, array([-2.28086224, -3.55746267, -4.15795714])), (0, array([ 4.91817719, -1.61207258,  4.08918334])), (0, array([-0.20628498,  1.93115185,  0.31740991])), (0, array([-2.96573499,  2.11102717,  2.02907233]))], [(1, array([ 1.43230177, -0.68345961, -1.46634662])), (1, array([ 0.68186651, -1.97819621,  4.34620946])), (1, array([-0.982763  , -2.64589012, -1.28628328])), (1, array([ 1.21245827,  1.39992696, -1.80400278])), (1, array([ 3.30515875,  3.94886303,  0.96699062]))], [(2, array([ 0.15201938,  1.39794141, -3.72252959])), (2, array([ 4.6616333 , -0.22642883, -2.93849021])), (2, array([-3.95199611, -4.23153409,  3.33087678])), (2, array([-1.16195152,  1.5747052 , -3.6826684 ])), (2, array([-0.68810805,  3.51427409,  1.07432477]))], [(3, array([-3.10383017, -0.54272962, -4.46022335])), (3, a